In [1]:

from torch import nn
import numpy as np

# todo

sddr single:
input to NN cannot be dicts


SDDR: may be we make our own get_parameters?

In [827]:


class Sddr_Single(nn.Module):
    
    def __init__(self, deep_models_dict, deep_shapes, struct_shapes, P):
        """
        deep_models_dict: dictionary where key are names of deep models and values are objects that define the deep models
        struct_shapes: number of structural features
        P: numpy matrix for the smoothing regularization (with added zero matrix in the beginning for the linear part)
        
        """
        super(Sddr_Single, self).__init__()
        self.P = P
        self.deep_models_dict = deep_models_dict
        
        #register external neural networks
        for key, value in deep_models_dict.items():
            self.add_module(key,value)
        
        
        self.structured_head = nn.Linear(struct_shapes,1, bias = False)
        
        if len(deep_models_dict) != 0:
            output_size_of_deep_models  = sum([deep_shapes[key] for key in deep_shapes.keys()])
            self.deep_head = nn.Linear(output_size_of_deep_models,1, bias = False)
            self._deep_models_exist = True
        else:
            self._deep_models_exist = False
        
              
        
    def _orthog_layer(self, Q, Uhat):
        """
        Utilde = Uhat - QQTUhat
        """
        Projection_Matrix = Q @ Q.T
        Utilde = Uhat - Projection_Matrix @ Uhat
        
        return Utilde
    
    
    def forward(self, datadict):
        """Comment 6.8.2020 We checked that we can actually have a dictionary as an input here. that should work fine"""
        
        X = datadict["structured"]
        
        if self._deep_models_exist:
            Q, R = torch.qr(X)

            Uhat_list = []
            for key in self.deep_models_dict.keys(): #assume that the input for the NN has the name of the NN as key
                net = self.deep_models_dict[key]
                Uhat_list.append(net(datadict[key]))

            Uhat = torch.cat(Uhat_list, dim = 1) #concatenate the outputs of the deep NNs

            Utilde = self._orthog_layer(Q, Uhat)
            
            deep_pred = self.deep_head(Utilde)
        else:
            deep_pred = 0
        
        structured_pred = self.structured_head(X)
        
        pred = structured_pred + deep_pred
        
        return pred
    
    def get_regularization(self):
        P = torch.from_numpy(self.P).float() # should have shape struct_shapes x struct_shapes, numpy array
        weights = self.structured_head.weight #should have shape 1 x struct_shapes
        
        
        regularization = weights @ P @ weights.T
        
        return regularization
        
        
        
class Sddr(nn.Module):
    
    def __init__(self, family, regularization_params, parsed_formula_contents):
        """
        family: string e.g. "gaussian", "binomial"...
        regularization_params: smoothing parameters
        parsed_formula_contents: dictionary with keys being parameters of the distribution, e.g. "eta" and "scale"
        and values being dicts with keys deep_models_dict, struct_shapes and P (as used in Sddr_Single)
        """
        super(Sddr, self).__init__()
        self.family = family
        self.regularization_params = regularization_params
        self.parameter_names = parsed_formula_contents.keys
        self.single_parameter_sddr_list = dict()
        for key, value in parsed_formula_contents.items():
            deep_models_dict = value["deep_models_dict"]
            deep_shapes = value["deep_shapes"]
            struct_shapes = value["struct_shapes"]
            P = value["P"]
            self.single_parameter_sddr_list[key] = Sddr_Single(deep_models_dict, deep_shapes, struct_shapes, P)
            
            #register the Sddr_Single network
            self.add_module(key,self.single_parameter_sddr_list[key])
                

        #define distributional layer
        if family == "normal":
            self.distribution_layer_type = torch.distributions.normal.Normal
    
    def _distribution_trafos(self,pred):
        #applies the specific transformations to the prediction so they they correspond to the restrictions
        #of the parameters
        #this is family specific
        pred_trafo = dict()
        add_const = 1e-8
        
        family = self.family
        if family == "normal":
            pred_trafo["loc"] = pred["loc"]
            pred_trafo["scale"] = add_const + pred["scale"].exp()
        
        return pred_trafo
    
    def forward(self,meta_datadict):
        
        pred = dict()
        for parameter_name, data_dict  in meta_datadict.items():
            print(parameter_name)
            sddr_net = self.single_parameter_sddr_list[parameter_name]
            pred[parameter_name] = sddr_net(data_dict)
            
        predicted_parameters = self._distribution_trafos(pred)
        
        #define distributional layer (takes eta and scale)
        self.distribution_layer = self.distribution_layer_type(**predicted_parameters)
        
        return self.distribution_layer
    
    def get_loss(self, Y):
    
        regul = 0
        for parameter_name, data_dict  in meta_datadict.items():
            sddr_net = self.single_parameter_sddr_list[parameter_name]
            regul += sddr_net.get_regularization()*self.regularization_params[parameter_name]
        log_loss = -self.distribution_layer.log_prob(Y)
        loss = log_loss + regul
        
        return loss
        
        


# test Sddr_Single

In [828]:
deep_shapes = {"dm1" : 5, "dm2" : 8}

net2= nn.Sequential(nn.Linear(10,3),nn.ReLU(), nn.Linear(3,8))

In [829]:
deep_models_dict = {"dm1" : nn.Linear(10,5, bias = False), "dm2" : net2}

In [830]:
struct_shapes = 10

In [831]:
P = np.eye(10)

In [832]:
data = torch.ones(20,10)
datadict = {"structured": data, "dm1": data, "dm2": data}

In [833]:
net = Sddr_Single(deep_models_dict, deep_shapes, struct_shapes, P)

In [834]:
net(datadict)

tensor([[-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535],
        [-0.0535]], grad_fn=<AddBackward0>)

In [835]:
net.get_regularization()

tensor([[0.4237]], grad_fn=<MmBackward>)

In [836]:
for i in net.parameters():
    print(i)

Parameter containing:
tensor([[-0.0450,  0.3132, -0.2533, -0.0663,  0.1513,  0.1618, -0.1711,  0.1751,
          0.2034, -0.1013],
        [ 0.1457, -0.1887,  0.1578, -0.0177, -0.0884, -0.2482,  0.1933,  0.1885,
          0.1223,  0.1456],
        [-0.2872,  0.2873,  0.0221,  0.1267, -0.1926, -0.1228, -0.1013, -0.3115,
         -0.1070,  0.3092],
        [ 0.2323, -0.0799,  0.2225,  0.0531,  0.0059,  0.1509, -0.1401,  0.2687,
          0.1905, -0.2308],
        [-0.0035, -0.1833, -0.1211, -0.0225, -0.2246,  0.0049, -0.0644, -0.2636,
         -0.1963, -0.2150]], requires_grad=True)
Parameter containing:
tensor([[ 0.1504,  0.0733, -0.0326,  0.1949, -0.0519, -0.0607, -0.0694,  0.0783,
         -0.2417, -0.0261],
        [-0.2603, -0.0073,  0.2344, -0.2413, -0.1986, -0.1428, -0.2594,  0.0118,
         -0.1722,  0.1692],
        [ 0.0141, -0.2903,  0.2476, -0.2810, -0.0929, -0.2952,  0.1316, -0.0939,
          0.1434, -0.0977]], requires_grad=True)
Parameter containing:
tensor([0.2397, 0.13

In [837]:
for name in net.children():
    print(name)

Linear(in_features=10, out_features=5, bias=False)
Sequential(
  (0): Linear(in_features=10, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=8, bias=True)
)
Linear(in_features=10, out_features=1, bias=False)
Linear(in_features=13, out_features=1, bias=False)


# Test Sddr

In [838]:
family= "normal"


In [839]:
parsed_formula_contents = dict()
parsed_formula_contents["loc"] = {"deep_models_dict": deep_models_dict, "deep_shapes": deep_shapes, "struct_shapes": struct_shapes, "P": P}
parsed_formula_contents["scale"] = {"deep_models_dict": dict(), "deep_shapes": dict(), "struct_shapes": struct_shapes, "P": P}


In [840]:
regularization_params = dict()

regularization_params["loc"] = 1.
regularization_params["scale"] = 1.


In [841]:
meta_datadict = dict()

meta_datadict["loc"] = datadict
meta_datadict["scale"] = {"structured": data}


In [842]:
bignet = Sddr(family, regularization_params, parsed_formula_contents)

In [843]:
bignet(meta_datadict)

loc
scale


Normal(loc: torch.Size([20, 1]), scale: torch.Size([20, 1]))

In [844]:
for i in bignet.children():
    print(i)

Sddr_Single(
  (dm1): Linear(in_features=10, out_features=5, bias=False)
  (dm2): Sequential(
    (0): Linear(in_features=10, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=8, bias=True)
  )
  (structured_head): Linear(in_features=10, out_features=1, bias=False)
  (deep_head): Linear(in_features=13, out_features=1, bias=False)
)
Sddr_Single(
  (structured_head): Linear(in_features=10, out_features=1, bias=False)
)


In [845]:
for i in bignet.parameters():
    print(i)

Parameter containing:
tensor([[-0.0450,  0.3132, -0.2533, -0.0663,  0.1513,  0.1618, -0.1711,  0.1751,
          0.2034, -0.1013],
        [ 0.1457, -0.1887,  0.1578, -0.0177, -0.0884, -0.2482,  0.1933,  0.1885,
          0.1223,  0.1456],
        [-0.2872,  0.2873,  0.0221,  0.1267, -0.1926, -0.1228, -0.1013, -0.3115,
         -0.1070,  0.3092],
        [ 0.2323, -0.0799,  0.2225,  0.0531,  0.0059,  0.1509, -0.1401,  0.2687,
          0.1905, -0.2308],
        [-0.0035, -0.1833, -0.1211, -0.0225, -0.2246,  0.0049, -0.0644, -0.2636,
         -0.1963, -0.2150]], requires_grad=True)
Parameter containing:
tensor([[ 0.1504,  0.0733, -0.0326,  0.1949, -0.0519, -0.0607, -0.0694,  0.0783,
         -0.2417, -0.0261],
        [-0.2603, -0.0073,  0.2344, -0.2413, -0.1986, -0.1428, -0.2594,  0.0118,
         -0.1722,  0.1692],
        [ 0.0141, -0.2903,  0.2476, -0.2810, -0.0929, -0.2952,  0.1316, -0.0939,
          0.1434, -0.0977]], requires_grad=True)
Parameter containing:
tensor([0.2397, 0.13

In [846]:
Y = torch.ones([20,1])

In [847]:
Y.size()

torch.Size([20, 1])

In [848]:
loss = bignet.get_loss(Y)

In [850]:
loss.mean().backward()

# testing for dictionary as input to neural network module

In [9]:
import torch

In [171]:
class testnn(nn.Module):
    
    def __init__(self):
        super(testnn, self).__init__()
        
        
        self.weights = nn.Parameter(torch.ones(784, 10) )
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, xa):
        #xa = datadict["a"]
        #xb = datadict["b"]
        out = xa @ self.weights + xa @ self.weights + self.bias
        return out
  

In [172]:
testnn_obj = testnn()

In [173]:
data = torch.ones([784,10]).T

In [174]:
out = testnn_obj(data)

In [175]:
param_list = []
for i in testnn_obj.parameters():
    param_list.append(i)

In [176]:
loss  = (out-100)**2

loss.mean().backward()

In [177]:
param_list[0].grad


tensor([[587.2000, 587.2000, 587.2000,  ..., 587.2000, 587.2000, 587.2000],
        [587.2000, 587.2000, 587.2000,  ..., 587.2000, 587.2000, 587.2000],
        [587.2000, 587.2000, 587.2000,  ..., 587.2000, 587.2000, 587.2000],
        ...,
        [587.2000, 587.2000, 587.2000,  ..., 587.2000, 587.2000, 587.2000],
        [587.2000, 587.2000, 587.2000,  ..., 587.2000, 587.2000, 587.2000],
        [587.2000, 587.2000, 587.2000,  ..., 587.2000, 587.2000, 587.2000]])

In [170]:
testnn_obj.zero_grad()